In [1]:
import sys
from common.path_utils import get_src_path, get_data_path
sys.path.append(str(get_src_path()))


from common.datatypes import ForecastingQuestion_stripped, ForecastingQuestion
import json

# llm_forecasting imports
from forecasters.llm_forecasting.prompts.prompts import PROMPT_DICT
from forecasters.llm_forecasting.utils.time_utils import get_todays_date, subtract_days_from_date
from forecasters.llm_forecasting import ranking, summarize, ensemble



/local/home/dpaleka/miniconda3/envs/consistency/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/local/home/dpaleka/miniconda3/envs/consistency/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### Load Data

In [2]:
data = []
with open(get_data_path() / "fq/real/questions_cleaned_formatted.jsonl", "r") as file:
    for line in file:
        data.append(json.loads(line))

In [3]:
sample_question = data[0]

In [4]:
fq = ForecastingQuestion(**sample_question)

### Retrieval

In [5]:
RETRIEVAL_CONFIG = {
    "NUM_SEARCH_QUERY_KEYWORDS": 3,
    "MAX_WORDS_NEWSCATCHER": 5,
    "MAX_WORDS_GNEWS": 8,
    "SEARCH_QUERY_MODEL_NAME": "gpt-4-1106-preview",
    "SEARCH_QUERY_TEMPERATURE": 0.0,
    "SEARCH_QUERY_PROMPT_TEMPLATES": [
        PROMPT_DICT["search_query"]["0"],
        PROMPT_DICT["search_query"]["1"],
    ],
    "NUM_ARTICLES_PER_QUERY": 5,
    "SUMMARIZATION_MODEL_NAME": "gpt-3.5-turbo-1106",
    "SUMMARIZATION_TEMPERATURE": 0.2,
    "SUMMARIZATION_PROMPT_TEMPLATE": PROMPT_DICT["summarization"]["9"],
    "NUM_SUMMARIES_THRESHOLD": 10,
    "PRE_FILTER_WITH_EMBEDDING": True,
    "PRE_FILTER_WITH_EMBEDDING_THRESHOLD": 0.32,
    "RANKING_MODEL_NAME": "gpt-3.5-turbo-1106",
    "RANKING_TEMPERATURE": 0.0,
    "RANKING_PROMPT_TEMPLATE": PROMPT_DICT["ranking"]["0"],
    "RANKING_RELEVANCE_THRESHOLD": 4,
    "RANKING_COSINE_SIMILARITY_THRESHOLD": 0.5,
    "SORT_BY": "date",
    "RANKING_METHOD": "llm-rating",
    "RANKING_METHOD_LLM": "title_250_tokens",
    "NUM_SUMMARIES_THRESHOLD": 20,
    "EXTRACT_BACKGROUND_URLS": True,
}

In [6]:
question = fq.title
background_info = fq.metadata["background_info"]
resolution_criteria = fq.body # resolution criteria and other info is in |body|

today_date = get_todays_date()
# If open date is set in data structure, change beginning of retrieval to question open date.
# Retrieve from [today's date - 1 month, today's date].
retrieval_dates = (
    subtract_days_from_date(today_date, 30),
    today_date,
)

In [7]:
(
    ranked_articles,
    all_articles,
    search_queries_list_gnews,
    search_queries_list_nc,
) = await ranking.retrieve_summarize_and_rank_articles(
    question,
    background_info,
    resolution_criteria,
    retrieval_dates,
    urls=[],
    config=RETRIEVAL_CONFIG,
    return_intermediates=True,
)

An error occurred while fetching the article: Article `download()` failed with Website protected with PerimeterX, url: None on URL https://news.google.com/rss/articles/CBMiU2h0dHBzOi8vd3d3Lm15c2FuYW50b25pby5jb20vYnVzaW5lc3MvYXJ0aWNsZS9zcGFjZXgtaHVtYW4tc3BhY2VmbGlnaHQtMTk0MzA2OTUucGhw0gEA?oc=5&hl=en-US&gl=US&ceid=US:en


In [10]:
all_summaries = summarize.concat_summaries(
    ranked_articles[: RETRIEVAL_CONFIG["NUM_SUMMARIES_THRESHOLD"]]
)

print(all_summaries[:3000], "...")

---
ARTICLES
[1] SpaceX making progress on Starship in-space refueling technologies (published on 2024-04-27)
Summary: NASA says SpaceX is on track to demonstrate in-space refueling of Starship next year, a critical technology for returning humans to the lunar surface. The next major milestone is a demonstration planned for 2025 where two Starships will dock in orbit, with one transferring propellants to the other. Propellant transfer technology is essential to SpaceX’s plans for Starship missions beyond low Earth orbit, including the Human Landing System (HLS) version of Starship that will be used to land astronauts on the moon starting with the Artemis 3 mission, currently scheduled for no earlier than September 2026. Elon Musk has noted that if SpaceX "gets lucky and things go according to plan", a manned flight could launch in the 2024 window with a landing on Mars in 2025.

[2] To the Stars: The Making of a Multiplanetary Species with SpaceX. (published on 2024-04-22)
Summary: Spa

### Reasoning

In [11]:
REASONING_CONFIG = {
    "BASE_REASONING_MODEL_NAMES": ["gpt-4-1106-preview", "gpt-4-1106-preview"],
    "BASE_REASONING_TEMPERATURE": 1.0,
    "BASE_REASONING_PROMPT_TEMPLATES": [
        [
            PROMPT_DICT["binary"]["scratch_pad"]["1"],
            PROMPT_DICT["binary"]["scratch_pad"]["2"],
        ],
        [
            PROMPT_DICT["binary"]["scratch_pad"]["new_3"],
            PROMPT_DICT["binary"]["scratch_pad"]["new_6"],
        ],
    ],
    "AGGREGATION_METHOD": "meta",
    "AGGREGATION_PROMPT_TEMPLATE": PROMPT_DICT["meta_reasoning"]["0"],
    "AGGREGATION_TEMPERATURE": 0.2,
    "AGGREGATION_MODEL_NAME": "gpt-4",
    "AGGREGATION_WEIGTHTS": None,
}

In [12]:
close_date = "N/A"  # data doesn't have explicit close date, so set to N/A
today_to_close_date = [today_date, close_date]

ensemble_dict = await ensemble.meta_reason(
    question=question,
    background_info=background_info,
    resolution_criteria=resolution_criteria,
    today_to_close_date_range=today_to_close_date,
    retrieved_info=all_summaries,
    reasoning_prompt_templates=REASONING_CONFIG["BASE_REASONING_PROMPT_TEMPLATES"],
    base_model_names=REASONING_CONFIG["BASE_REASONING_MODEL_NAMES"],
    base_temperature=REASONING_CONFIG["BASE_REASONING_TEMPERATURE"],
    aggregation_method=REASONING_CONFIG["AGGREGATION_METHOD"],
    weights=REASONING_CONFIG["AGGREGATION_WEIGTHTS"],
    meta_model_name=REASONING_CONFIG["AGGREGATION_MODEL_NAME"],
    meta_prompt_template=REASONING_CONFIG["AGGREGATION_PROMPT_TEMPLATE"],
    meta_temperature=REASONING_CONFIG["AGGREGATION_TEMPERATURE"],
)

In [13]:
print("REASONING\n", "=================")
print(ensemble_dict["meta_reasoning"])

print("PROBABILITY\n", "=================")
print(ensemble_dict["meta_prediction"])

REASONING
1. Reasons why the answer might be no:
- Technical Challenges: SpaceX is currently facing a 50% performance shortfall with the Starship meant for lunar missions. This could potentially delay Mars missions if the issue is not resolved in a timely manner (Article 3).
- Testing and Development Delays: The aggressive timeline depends on successful tests and rapid development of technologies like in-space refueling. Any delays in these areas could push back the manned Mars mission (Article 1).
- Launch Window Constraints: With launch windows to Mars occurring roughly every two years, any significant delay could miss a critical window, thereby pushing the mission beyond 2029 (Article 4).
- Regulatory and Safety Concerns: Human space missions are subject to rigorous safety standards and approvals. Obtaining the necessary clearance may take longer than anticipated (Implied from all articles).
- Budget and Funding: The cost and funding of such ambitious space missions can be substanti

### Testing "Advanced Forecaster"

In [14]:
from forecasters.advanced_forecaster import AdvancedForecaster
af = AdvancedForecaster()

In [15]:
final_prob = await af.call_async(sentence=fq)

An error occurred while fetching the article: Article `download()` failed with Website protected with PerimeterX, url: None on URL https://news.google.com/rss/articles/CBMiU2h0dHBzOi8vd3d3Lm15c2FuYW50b25pby5jb20vYnVzaW5lc3MvYXJ0aWNsZS9zcGFjZXgtaHVtYW4tc3BhY2VmbGlnaHQtMTk0MzA2OTUucGhw0gEA?oc=5&hl=en-US&gl=US&ceid=US:en


In [16]:
print("Final LLM probability", final_prob)

Final LLM probability 0.6
